In [1]:
import os
from processing import split_train_test

In [2]:
filepaths = ['data/airplane.ndjson', 'data/apple.ndjson', 'data/umbrella.ndjson']
for filepath in filepaths:
    split_train_test(filepath, 80, 20)

created path: images/train/airplane
created path: images/test/airplane
151623 images processed:
+ 108663 selected to train
+ 27160 selected to test
 + 15800 skipped
created path: images/train/apple
created path: images/test/apple
144722 images processed:
+ 111922 selected to train
+ 27980 selected to test
 + 4820 skipped
created path: images/train/umbrella
created path: images/test/umbrella
124084 images processed:
+ 96414 selected to train
+ 24100 selected to test
 + 3570 skipped


In [7]:
classnames = ['airplane', 'apple', 'umbrella']
for classname in classnames:
    classdir = './images/train/{}'.format(classname)
    print('total {} images: {}'.format(classname, len(os.listdir(classdir))))

total airplane images: 108663
total apple images: 111922
total umbrella images: 96414


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
batch_size = 32

train_datagen = ImageDataGenerator()
train_flow = train_datagen.flow_from_directory(
    './images/train', # source category directory
    batch_size = batch_size,
    classes = classnames,
    class_mode = 'categorical')

Found 316999 images belonging to 3 classes.


In [16]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [26]:
input_layer = Input(shape=(256, 256, 3))

convolution_layers = [
    {
        'output': 16,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 32,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
    {
        'output': 64,
        'k_size': (3, 3),
        'active': 'relu',
        'pool_size': 2,
        'stride': 2,
    },
]

current_layer = input_layer

for layer in convolution_layers:
    current_layer = Conv2D(layer['output'], layer['k_size'], activation=layer['active'])(current_layer)
    current_layer = MaxPooling2D(layer['pool_size'], layer['stride'])(current_layer)

current_layer = Flatten()(current_layer)
current_layer = Dense(128, activation='relu')(current_layer)

output_layer = Dense(len(classnames), activation='softmax')(current_layer)
model = Model(inputs=input_layer, outputs=output_layer)


In [27]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 254, 254, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 125, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 60, 60, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 30, 30, 64)        0   

In [28]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['acc'])

In [ ]:
n_epochs = 30
history = model.fit_generator(
    train_flow,
    steps_per_epoch=total_sample // batch_size,
    epochs=n_epochs,
    verbose=1,
)

Epoch 1/30
